# Data Preprocessing before ML
## Import data

In [87]:
import os
os.chdir('C:/Sereda/Lectures/Springboard/Capstone2_CustomerChurn/')
import pandas as pd
import numpy as np
from pathlib import Path

mt = pd.read_csv('data/mt.csv', parse_dates=['registration_init_time','transaction_date'], infer_datetime_format = True, index_col=None)

## Cut-off date
Since the prediction is made at a specific time (usually the end of a month), we need to disregard all future data before the model training and testing.

In [88]:
#cutoff_date = latest_date
cutoff_date = pd.Timestamp('2016-10-22')
print("Applying cutoff time", cutoff_date.strftime('%Y-%m-%d'))
mt_cut = mt.loc[mt.transaction_date <= cutoff_date]
print("No. of records:", len(mt_cut))
member_list = mt_cut['msno'].unique() # list of all member id's
print("No. of members:", len(member_list))
mt.head()

Applying cutoff time 2016-10-22
No. of records: 17298
No. of members: 2085


,msno,city,bd,gender,registered_via,registration_init_time,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel,discount,is_churn,registration_weekday
0,0Av7bk+3pPmkXj4ZwXFvO1xGdyTXHdFC2rbl8RTcHik=,13.0,28.0,female,9.0,2004-03-27,32,410,1788,1788,0,2015-06-16,2016-07-30,0,0,0,5
1,0Av7bk+3pPmkXj4ZwXFvO1xGdyTXHdFC2rbl8RTcHik=,13.0,28.0,female,9.0,2004-03-27,38,30,149,149,0,2016-08-01,2016-08-31,0,0,0,5
2,0Av7bk+3pPmkXj4ZwXFvO1xGdyTXHdFC2rbl8RTcHik=,13.0,28.0,female,9.0,2004-03-27,38,30,149,149,0,2016-09-13,2016-10-13,0,0,0,5
3,0Av7bk+3pPmkXj4ZwXFvO1xGdyTXHdFC2rbl8RTcHik=,13.0,28.0,female,9.0,2004-03-27,32,415,1599,1599,0,2016-10-23,2017-12-12,0,0,1,5
4,9N+q58Vl33+QW6zHwivroNlgJw88g9ghwX/ke+OhgtM=,15.0,0.0,female,9.0,2004-03-27,34,0,0,149,1,2015-03-31,2015-04-30,0,-149,1,5


## Select only first `n_trans` transactions for each member
Cut-off and grouping can also be performed based on a constant number of transactions across all customers in order to predict churn for a customer who had $n$ transactions.

In [89]:
n_trans=1
mt_cutn = pd.DataFrame({'msno':[],'city':[],'bd':[],'gender':[],'registered_via':[],'registration_init_time':[],'payment_method_id':[],'payment_plan_days':[],'plan_list_price':[],'actual_amount_paid':[],'is_auto_renew':[],'transaction_date':[],'membership_expire_date':[],'is_cancel':[],'discount':[],'is_churn':[]})
member_list = mt_cut['msno'].unique()
for member_id in member_list:
    member_transactions = mt_cut.loc[mt_cut['msno'] == member_id]
    if len(member_transactions) >= n_trans:
        tr = member_transactions.head(n_trans)
        #mt_cut1.loc[len(mt_cut1.index)] = [tr['msno'],tr['city'].iloc[0],0,0,0,0,0,0,0,0,0,0,0,0,0]n
        mt_cutn = mt_cutn.append(tr, ignore_index = True)
print(mt_cutn.shape)
mt_cutn.head()

(2085, 17)


,msno,city,bd,gender,registered_via,registration_init_time,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel,discount,is_churn,registration_weekday
0,0Av7bk+3pPmkXj4ZwXFvO1xGdyTXHdFC2rbl8RTcHik=,13.0,28.0,female,9.0,2004-03-27,32.0,410.0,1788.0,1788.0,0.0,2015-06-16,2016-07-30,0.0,0.0,0.0,5.0
1,9N+q58Vl33+QW6zHwivroNlgJw88g9ghwX/ke+OhgtM=,15.0,0.0,female,9.0,2004-03-27,34.0,0.0,0.0,149.0,1.0,2015-03-31,2015-04-30,0.0,-149.0,1.0,5.0
2,9rzbu7TiCDnfkSXSx9O0DZhmEBSncZ0VBmS2l1b5kg8=,22.0,34.0,male,9.0,2004-03-27,37.0,31.0,149.0,149.0,1.0,2015-01-11,2015-02-12,0.0,0.0,0.0,5.0
3,/G/XFknHm94YVoCefhCyvnB9mAFM0rUSMqSNA/n+jTg=,13.0,34.0,male,9.0,2004-04-07,34.0,0.0,0.0,149.0,1.0,2015-03-31,2015-04-30,0.0,-149.0,1.0,2.0
4,BvmNMrNw/X4PB8ueWGZ0JswZoUAGGpMZET03F+rkmTI=,4.0,34.0,male,9.0,2004-05-18,32.0,410.0,1788.0,1788.0,0.0,2016-01-04,2017-02-20,0.0,0.0,0.0,1.0


## Drop useless columns: member ID, plan_list_price, all dates

In [90]:
mt_cutn = mt_cutn.drop(['msno','plan_list_price','registration_init_time','transaction_date','membership_expire_date'],axis=1)
print(mt_cutn.shape)
mt_cutn.head()

(2085, 12)


,city,bd,gender,registered_via,payment_method_id,payment_plan_days,actual_amount_paid,is_auto_renew,is_cancel,discount,is_churn,registration_weekday
0,13.0,28.0,female,9.0,32.0,410.0,1788.0,0.0,0.0,0.0,0.0,5.0
1,15.0,0.0,female,9.0,34.0,0.0,149.0,1.0,0.0,-149.0,1.0,5.0
2,22.0,34.0,male,9.0,37.0,31.0,149.0,1.0,0.0,0.0,0.0,5.0
3,13.0,34.0,male,9.0,34.0,0.0,149.0,1.0,0.0,-149.0,1.0,2.0
4,4.0,34.0,male,9.0,32.0,410.0,1788.0,0.0,0.0,0.0,0.0,1.0


## Drop other columns for comparison

In [91]:
mt_cutn = mt_cutn.drop(['registration_weekday'],axis=1)
print(mt_cutn.shape)
mt_cutn.head()

(2085, 11)


,city,bd,gender,registered_via,payment_method_id,payment_plan_days,actual_amount_paid,is_auto_renew,is_cancel,discount,is_churn
0,13.0,28.0,female,9.0,32.0,410.0,1788.0,0.0,0.0,0.0,0.0
1,15.0,0.0,female,9.0,34.0,0.0,149.0,1.0,0.0,-149.0,1.0
2,22.0,34.0,male,9.0,37.0,31.0,149.0,1.0,0.0,0.0,0.0
3,13.0,34.0,male,9.0,34.0,0.0,149.0,1.0,0.0,-149.0,1.0
4,4.0,34.0,male,9.0,32.0,410.0,1788.0,0.0,0.0,0.0,0.0


## Data types

In [102]:
for col in ['city','bd','registered_via','payment_method_id','payment_plan_days','is_auto_renew','is_cancel','is_churn']:
    mt_cutn.loc[:,col] = mt_cutn[col].astype('int64') # convert data type
mt_cut.dtypes # verify

msno                              object
city                             float64
bd                               float64
gender                            object
registered_via                   float64
registration_init_time    datetime64[ns]
payment_method_id                  int64
payment_plan_days                  int64
plan_list_price                    int64
actual_amount_paid                 int64
is_auto_renew                      int64
transaction_date          datetime64[ns]
membership_expire_date            object
is_cancel                          int64
discount                           int64
is_churn                           int64
registration_weekday               int64
dtype: object

In [103]:
mt_cutn.head()

,city,bd,gender,registered_via,payment_method_id,payment_plan_days,actual_amount_paid,is_auto_renew,is_cancel,discount,is_churn
0,13,28,female,9,32,410,1788.0,0,0,0.0,0
1,15,0,female,9,34,0,149.0,1,0,-149.0,1
2,22,34,male,9,37,31,149.0,1,0,0.0,0
3,13,34,male,9,34,0,149.0,1,0,-149.0,1
4,4,34,male,9,32,410,1788.0,0,0,0.0,0


## One row per member

In [97]:
print("Reshape features to have one row per member")
# Column names for all features in all transactions
columns = list(['city','bd','gender','registered_via'])
for i in range(1,n_trans+1):
    for col in ['payment_method_id','payment_plan_days','actual_amount_paid','is_auto_renew','is_cancel','discount']:
        columns.append(col+str(i))
columns.append('is_churn')
#columns.append('registration_weekday')
print(columns)
mt_cutn_1row = pd.DataFrame(columns=columns)
for i in range(mt_cutn.shape[0]//n_trans): # members
    tr_all = mt_cutn.iloc[i * n_trans, 0:4] # common columns for all transactions
    for t in range(n_trans):
        tr_all = np.concatenate((tr_all, mt_cutn.iloc[i * n_trans + t, 4:10]), axis=0)
    tr_all = np.concatenate((tr_all, mt_cutn.iloc[(i+1) * n_trans - 1, 10:len(mt_cutn.columns)]), axis=0) # is_churn, added features
    tr_all = pd.DataFrame(tr_all, index=columns).transpose()
    mt_cutn_1row = mt_cutn_1row.append(tr_all)
mt_cutn_1row.head()

Reshape features to have one row per member
['city', 'bd', 'gender', 'registered_via', 'payment_method_id1', 'payment_plan_days1', 'actual_amount_paid1', 'is_auto_renew1', 'is_cancel1', 'discount1', 'is_churn']


,city,bd,gender,registered_via,payment_method_id1,payment_plan_days1,actual_amount_paid1,is_auto_renew1,is_cancel1,discount1,is_churn
0,13,28,female,9,32,410,1788.0,0,0,0.0,0
0,15,0,female,9,34,0,149.0,1,0,-149.0,1
0,22,34,male,9,37,31,149.0,1,0,0.0,0
0,13,34,male,9,34,0,149.0,1,0,-149.0,1
0,4,34,male,9,32,410,1788.0,0,0,0.0,0


## One-hot encoding
All categorical data are converted from numbers to 0/1 dummy variables.

In [105]:
df = pd.get_dummies(mt_cutn_1row, drop_first=False, columns=mt_cutn_1row.columns.drop('is_churn')) # do not hot-encode labels
print("No. of predictors:", df.shape[1]-1)
df.head()

No. of predictors: 162


,is_churn,city_0,city_1,city_3,city_4,city_5,city_6,city_7,city_8,city_9,...,discount1_-894.0,discount1_-799.0,discount1_-150.0,discount1_-149.0,discount1_-129.0,discount1_-50.0,discount1_0.0,discount1_20.0,discount1_30.0,discount1_149.0
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


## Drop unknown `is_churn`

In [106]:
df = df.loc[df['is_churn'] != 2] # drop unknown churn
df.head()

,is_churn,city_0,city_1,city_3,city_4,city_5,city_6,city_7,city_8,city_9,...,discount1_-894.0,discount1_-799.0,discount1_-150.0,discount1_-149.0,discount1_-129.0,discount1_-50.0,discount1_0.0,discount1_20.0,discount1_30.0,discount1_149.0
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


## Save ML-ready data

In [107]:
# Save data
#if not Path("./data/df.csv").is_file():
df.to_csv("./data/df.csv", index=False)

In [101]:
# Save data
#if not Path("./data/df.csv").is_file():
df.to_csv("./data/df_20161022_1.csv", index=False)